In [17]:
import os
import json
import tqdm
import pdf2image

In [5]:
dataset_path = 'dataset'
result_path = 'result'

In [ ]:
def sectorize_pdf(dir):
    files = os.listdir(dir)
    return [f for f in files if f.endswith('.pdf')]

pdf_files = sectorize_pdf(dataset_path)

In [ ]:
current_file = 0

result_data = os.path.join(result_path, 'data')
result_image = os.path.join(result_path, 'images')

os.makedirs(result_data, exist_ok=True)
os.makedirs(result_image, exist_ok=True)

for i, pdf_name in enumerate(tqdm.tqdm(pdf_files, desc='Processing PDFA files')):
    json_name = f'{os.path.splitext(pdf_name)[0]}.json'
    pdf_path = os.path.join(dataset_path, pdf_name)
    json_path = os.path.join(dataset_path, json_name)
    
    with open(json_path, 'r', encoding='utf-8') as j:
        data = json.load(j)

    images = pdf2image.convert_from_path(pdf_path, poppler_path=r"C:\Library\bin")

    for img, page in tqdm.tqdm(zip(images, data['pages']), desc=f'Processing {i} page'):
        txt_path = os.path.join(result_data, f'PDFA_{current_file}.txt')
        img_path = os.path.join(result_image, f'PDFA_{current_file}.png')

        with open(txt_path, 'w', encoding='utf-8') as txt:
            for text, box in zip(page['words']['text'], page['words']['bbox']):
                txt.write(f"{text} {box[0] + box[2] / 2} {box[1] + box[3] / 2} {box[2]} {box[3]}\n")

        img.save(img_path, 'PNG')
        current_file += 1